In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import torchvision.models as models
import torch.nn as nn
import torch
from torch import optim
import os
from torchvision.transforms import transforms
from PIL import Image
import cv2

checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/BAE-APA/model/checkpoint.pth'
image_path = '/content/drive/MyDrive/Colab Notebooks/BAE-APA/test/김치1.jpg'
image = cv2.imread(image_path)

labels = ['오렌지 주스',
              '우유',
              '샐러드',
              '계란',
              '고구마맛탕',
              '김치',
              '쌀밥',
              '콩나물',
              '치킨',
              '비빔밥',
              '돈가스']

def load_model(checkpoint_path, labels):
    num_classes = len(labels)
    model = models.resnet50()
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 101)
    classifier = nn.Linear(model.fc.out_features, num_classes)
    model = nn.Sequential(model,
                          classifier)
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    return model

def predict_food(model, image, labels):
    image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    num_classes = len(labels)
    transform = transforms.Compose([transforms.Resize(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])])
    image = transform(image)

    # Make a prediction with the model
    with torch.no_grad():
        output = model(image.unsqueeze(0))

    # Extract the predicted label
    _, predicted_idx = torch.max(output, 1)
    predicted_label = predicted_idx.item()
    predicted = labels[predicted_label]

    return predicted

In [28]:
model = load_model(checkpoint_path, labels)
predicted = predict_food(model, image, labels)
predicted

'김치'

In [5]:
import torchvision.models as models
import torch.nn as nn

model = models.resnet50()
num_features = model.fc.in_features
num_classes = 11

# 새로운 linear layer로 대체
model.fc = nn.Linear(num_features, 101)
model = nn.Sequential(model,
                      nn.Linear(model.fc.out_features, num_classes))

In [6]:
from torch import optim

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 5)

In [8]:
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/BAE-APA/model/checkpoint.pth'

checkpoint = torch.load(checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
optimizer.param_groups[0]['lr'] = checkpoint['learning_rate']

In [ ]:
model.eval()

In [10]:
import os

# Set the path to the directory containing the files
directory = '/content/drive/MyDrive/Colab Notebooks/BAE-APA/test/'

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.jfif'):
        # Create the new filename by replacing the extension with '.jpg'
        new_filename = os.path.join(directory, filename.replace('.jfif', '.jpg'))

        # Rename the file
        os.rename(os.path.join(directory, filename), new_filename)

In [19]:
from torchvision.transforms import transforms

transform = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])


In [21]:
labels = ['오렌지 주스',
 '우유',
 '샐러드',
 '계란',
 '고구마맛탕',
 '김치',
 '쌀밥',
 '콩나물',
 '치킨',
 '비빔밥',
 '돈가스']

In [22]:
from PIL import Image

for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        # Open the image file
        image_path = os.path.join(directory, filename)
        image = Image.open(image_path).convert("RGB")

        # Apply transformations to the image
        image = transform(image)

        # Make a prediction with the model
        with torch.no_grad():
            output = model(image.unsqueeze(0))

        # Extract the predicted label
        _, predicted_idx = torch.max(output, 1)
        predicted_label = predicted_idx.item()

        print(f"File: {filename}, Predicted Label: {labels[predicted_label]}")

File: 치킨1.jpg, Predicted Label: 치킨
File: 치킨2.jpg, Predicted Label: 치킨
File: 샐러드.jpg, Predicted Label: 샐러드
File: 샐러드2.jpg, Predicted Label: 샐러드
File: 김치1.jpg, Predicted Label: 김치
File: 콩나물.jpg, Predicted Label: 콩나물
File: 오렌지주스.jpg, Predicted Label: 오렌지 주스
